# Parsing Randomness
This is an example showing the principles of "Free Generators", a data structure both interpretable as:
- A Generator
- A Parser

First, like in the paper [TODO], the "original" Parser and Generator will be shown. Next, they will both be build using the introduced data structure `FreeGen`.

The utilised example are Binary Digit trees, first introduced here:

## Binary Digit trees
The trees will have the following elements:
- `N` stands for a node
  - A Node has 2 children (other tree)
  - and a Digit assigned (`0..9`)
- `L` stands for leaf

An example could look like this:
```haskell
N: 2 
|- L
`- N: 1
   |- N: 6 -> L L
   `- L
```

A **Generator** creates such a data structure at random. A **Parser** gets some String input (for the shown example: `N 2 L N 1 N 6 L L L`) and creates the according instance of the data structure.


In [88]:
:reload
:load ./Definitions/Tree.hs

import Definitions.Tree

## Implementation of original Generator and Parser
To show the analogy of the structure in Generators and Parsers, this section will start with the original implementation of both for creating (either making or parsing) a Binary Digit Tree.

Both the utilised Generator and Parser are already existing implementations from other libraries.

In [89]:
import Test.QuickCheck (Gen)
import qualified Test.QuickCheck.Gen as Gen

generateTree :: Int -> Gen Tree
generateTree 0 = return Leaf
generateTree h = do
  c <- Gen.frequency [(1, return 'L'), (3, return 'N')]
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- generateInt
      l <- generateTree (h-1)
      r <- generateTree (h-1)
      return (Node x l r)

generateInt = Gen.oneof [ return x | x<-[0..9] ]

Gen.generate (generateTree 4)


L

In [90]:
import Text.Parsec.String (Parser)
import Text.Parsec
import Data.Char

parseTree :: Int -> Parser Tree
parseTree 0 = return Leaf
parseTree h = do
  c <- oneOf "LN"
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- parseInt
      l <- parseTree (h-1)
      r <- parseTree (h-1)
      return (Node x l r)

parseInt = digitToInt <$> digit

parse (parseTree 5) "" "N2LN1N6LLL"

Right 
N: 2
|- L
`- N: 1
   |- N: 6 -> L L
   `- L

## Implementation of Free Generator
As a first step, the general Free Generator for the Tree exampale will be defined. 

Important to notice: the resulting output `FreeGen Tree` is simply an abstract data structure. Without the according interpretation, it is neither a generator nor a parser. Still, the structure has a remarkable resemblence with both the definition to get a `Gen Tree` and a `Parser Tree`.

In [91]:
:reload
:load ./Definitions/FreeGen.hs

import Definitions.FreeGen

In [92]:
freeGenTree :: Int -> FreeGen Tree
freeGenTree 0 = Return Leaf
freeGenTree h = do
  c <- pick [(1, 'L', return 'L'), (3, 'N', return 'N')]
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- freeGenInt
      l <- freeGenTree (h-1)
      r <- freeGenTree (h-1)
      return (Node x l r)

freeGenInt = pick [(1, intToDigit x, return x) | x<-[0..9]]

freeGen = freeGenTree 4

## Interpretations for FreeGen

Now follows the interesting part about the created `FreeGen Tree`: the different interpretations. The results should be
- a Generator (`Gen Tree`)
- a Parser (`Parser Tree`)

In [93]:
gen = interpretAsG freeGen
Gen.generate gen


N: 3
|- N: 9 -> L L
`- N: 2
   |- N: 8
   |  |- N: 9 -> L L
   |  `- N: 3 -> L L
   `- N: 8
      |- N: 3 -> L L
      `- N: 0 -> L L

In [94]:
parser = interpretAsP freeGen
parse parser "" "N2LN1N6LLL"

Right 
N: 2
|- L
`- N: 1
   |- N: 6 -> L L
   `- L

From the two upper code segments it can be shown, that both the interpretation as a generetor and parser work. 

To show that a generator can now be indeed separated into a parser and a generator for random sequences, an additional interpretation needs to be used:
- Randomness interpretation (`Gen String`)

With the resulting randomness generator (that creates random Strings), the generator can then be factored into the parser and the distribution over choice sequences.

In [95]:
random = interpretAsR freeGen

Gen.generate (parse parser "" <$> random)

Right 
N: 6
|- N: 0
|  |- L
|  `- N: 8
|     |- N: 1 -> L L
|     `- N: 1 -> L L
`- N: 2
   |- N: 7
   |  |- N: 6 -> L L
   |  `- N: 4 -> L L
   `- N: 5
      |- N: 1 -> L L
      `- N: 3 -> L L